In [2]:
%load_ext dotenv
%dotenv

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

In [4]:
search.invoke("What is the weather in Da Nang?")

[{'url': 'https://www.weatherapi.com/',
  'content': "Weather in Da Nang is {'location': {'name': 'Da Nang', 'region': '', 'country': 'Vietnam', 'lat': 21.72, 'lon': 105.35, 'tz_id': 'Asia/Bangkok', 'localtime_epoch': 1708493676, 'localtime': '2024-02-21 12:34'}, 'current': {'last_updated_epoch': 1708493400, 'last_updated': '2024-02-21 12:30', 'temp_c': 29.6, 'temp_f': 85.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 94, 'wind_dir': 'E', 'pressure_mb': 1008.0, 'pressure_in': 29.76, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 52, 'cloud': 0, 'feelslike_c': 31.1, 'feelslike_f': 87.9, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 7.0, 'gust_mph': 4.4, 'gust_kph': 7.1}}"},
 {'url': 'https://www.weathertab.com/en/c/2024/02/socialist-republic-of-vietnam/thanh-pho-da-nang/da-nang/',
  'content': 'Free Long Range Weather Forecast for Da Nang, Thành Phố Đà Nẵng February 2024.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/evaluation/faq/datasets-client")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [6]:
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [7]:
retriever.get_relevant_documents("How to upload a dataset")

[Document(page_content='from CSV\u200bIn this section, we will demonstrate how you can create a dataset by uploading a CSV file.First, ensure your CSV file is properly formatted with columns that represent your input and output keys. These keys will be utilized to map your data properly during the upload. You can specify an optional name and description for your dataset. Otherwise, the file name will be used as the dataset name and no description will be provided.Python SDKTypeScript SDKfrom langsmith import Clientimport osos.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"os.environ["LANGCHAIN_API_KEY"] = "<YOUR-LANGSMITH-API-KEY>"client = Client()csv_file = \'path/to/your/csvfile.csv\'input_keys = [\'column1\', \'column2\'] # replace with your input column namesoutput_keys = [\'output1\', \'output2\'] # replace with your output column namesdataset = client.upload_csv(    csv_file=csv_file,    input_keys=input_keys,    output_keys=output_keys,    name="My CSV Dataset"

In [8]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about Langsmith."
)

In [9]:
tools = [search, retriever_tool]

## Create the agent

In [10]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
from langchain import hub

# Get the prompt template to use
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [12]:
from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [13]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## Run the Agent

In [14]:
agent_executor.invoke({"input": "Hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'Hi!', 'output': 'Hello! How can I assist you today?'}

In [35]:
agent_executor.invoke({"input": "How can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith testing'}`


How to manage datasets programmatically | 🦜️🛠️ LangSmith

Skip to main content🦜️🛠️ LangSmith DocsLangChain Python DocsLangChain JS/TS DocsLangSmith API DocsSearchGo to AppLangSmithUser GuideSetupPricing (Coming Soon)Self-HostingTracingEvaluationOverviewQuick StartConceptsHow-To GuidesHow to manage datasets in the appHow to create datasets from the clientHow to list datasets from the clientHow to list datapoints from the clientHow to create custom evaluatorsHow to use off-the-shelf LangChain evaluatorsHow to paraphrase examples to expand your datasetHow to create new examples using promptingRecommendationsMonitoringPrompt HubEvaluationHow-To GuidesHow to create datasets from the clientOn this pageHow to manage datasets programmaticallyYou can create a dataset from existing runs or upload a CSV file (or pandas dataframe in python).Once you have a dataset created, you can continue

{'input': 'How can langsmith help with testing?',
 'output': 'Langsmith can help with testing by allowing you to manage datasets programmatically. You can create datasets from existing runs or upload CSV files or pandas dataframes in Python. Once you have a dataset created, you can continue to add new runs to it as examples. It is recommended to organize datasets to target a single task, usually served by a single chain or LLM. This can help in testing and evaluating models efficiently.'}

In [15]:
agent_executor.invoke({"input": "What is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "Weather in San Francisco is {'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1708493874, 'localtime': '2024-02-20 21:37'}, 'current': {'last_updated_epoch': 1708493400, 'last_updated': '2024-02-20 21:30', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 3.8, 'wind_kph': 6.1, 'wind_degree': 40, 'wind_dir': 'NE', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.1, 'precip_in': 0.0, 'humidity': 93, 'cloud': 75, 'feelslike_c': 10.5, 'feelslike_f': 51.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 7.1, 'gust_kph': 11.4}}"}, {'url': 'https:/

{'input': 'What is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 52.0°F (11.1°C). The wind is coming from the northeast at 6.1 km/h, and the humidity is at 93%.'}

## Adding the memory

In [17]:
# Here we pass in an empty list of messages for chat_history because it is the first message in the chat
agent_executor.invoke({"input": "hi! my name is Tran", "chat_history": []})



> Entering new AgentExecutor chain...
Hello Tran! How can I assist you today?

> Finished chain.


{'input': 'hi! my name is Tran',
 'chat_history': [],
 'output': 'Hello Tran! How can I assist you today?'}

In [18]:
from langchain_core.messages import AIMessage, HumanMessage

In [19]:
agent_executor.invoke({
    "chat_history": [
        HumanMessage(content="hi! my name is Tran"),
        AIMessage(content="Hello Tran! How can I assist you today?"),
    ],
    "input": "What's my  name?",
})



> Entering new AgentExecutor chain...
Your name is Tran. How can I assist you today, Tran?

> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is Tran'),
  AIMessage(content='Hello Tran! How can I assist you today?')],
 'input': "What's my  name?",
 'output': 'Your name is Tran. How can I assist you today, Tran?'}

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [21]:
message_history = ChatMessageHistory()

In [23]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [24]:
agent_with_chat_history.invoke(
    {
        "input": "Hi! I'm Tran"
    },
    config={
        "configurable": {"session_id": "<foo>"}
    }
)



> Entering new AgentExecutor chain...
Hello Tran! How can I assist you today?

> Finished chain.


{'input': "Hi! I'm Tran",
 'chat_history': [],
 'output': 'Hello Tran! How can I assist you today?'}

In [25]:
agent_with_chat_history.invoke(
    {"input": "what's my name?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Tran. How can I assist you today, Tran?

> Finished chain.


{'input': "what's my name?",
 'chat_history': [HumanMessage(content="Hi! I'm Tran"),
  AIMessage(content='Hello Tran! How can I assist you today?')],
 'output': 'Your name is Tran. How can I assist you today, Tran?'}